# Santosh Ainumpudi -U68091846

Parkinson's disease is a serious neurological condition that requires accurate and timely diagnosis. Therefore, the primary goal of any model developed for Parkinson's disease diagnosis should be to maximize accuracy.
The dataset is a balanced dataset, meaning that there are an equal number of samples for both positive and negative classes. In such cases, accuracy is a reliable and appropriate metric to evaluate the performance of the model.
Accuracy provides a simple and easy-to-understand measure of the model's overall performance. It tells us the proportion of correctly classified instances to the total number of instances, which is a useful measure for assessing the model's effectiveness.We need to accurately predict the presence of Parkinson disease as 1 or 0 (i.e as Yes or No).

The choice of the best scoring metric for the Parkinson's disease dataset is that as it depends on the distribution of the target variable, and the relative importance of true positives and false positives or true negatives and false negatives.


Since goal is to accurately predict the presence or absence of Parkinson's disease, I am taking accuracy as the scoring metric.

# Fitting the Models

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix

np.random.seed(1)

# Load the Data

In [3]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")


# Modelling the Data 

# Logistic Regession using Random Search and Grid Search

# Random Search


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

score_measure = "accuracy"
kfolds = 3

param_grid = {'C':[0.001,0.01,0.1,1,10], # C is the regulization strength
               'penalty':['l1', 'l2','elasticnet','none'],
              'solver':['saga','liblinear'],
              'max_iter': np.arange(200,800)
    
    
}

lr = LogisticRegression()
rand_search = RandomizedSearchCV(estimator = lr, param_distributions=param_grid, cv=kfolds, n_iter=700,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train,y_train)

print(f"The {score_measure} score is {rand_search.best_score_} with {rand_search.best_params_} parameters")


bestlr = rand_search.best_estimator_

Fitting 3 folds for each of 700 candidates, totalling 2100 fits
The accuracy score is 0.8528180354267311 with {'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 381, 'C': 10} parameters


C:\Users\santo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
783 fits failed out of a total of 2100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
279 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\santo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\santo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\santo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 457, in _check_solver
    raise ValueError(


In [5]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

Accuracy=0.8305085 Precision=0.8541667 Recall=0.9318182 F1=0.8913043


# GridSearch

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
score_measure = "accuracy"
kfolds = 3
best_penality = rand_search.best_params_['penalty']
best_solver = rand_search.best_params_['solver']
min_regulization_strength=rand_search.best_params_['C']
min_iter = rand_search.best_params_['max_iter']

#Using the best parameters from  Random Search to perform the grid search
param_grid = {
    
    'C':np.arange(min_regulization_strength-1,min_regulization_strength+1), 
               'penalty':[best_penality],
              'solver':[best_solver],
              'max_iter': np.arange(min_iter-100,min_iter+100)
}

logreg =  LogisticRegression()
grid_search = GridSearchCV(estimator = logreg, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                return_train_score=True)

_ = grid_search.fit(X_train,y_train)

print(f"The {score_measure} score is {rand_search.best_score_} with parameters: {rand_search.best_params_}")

bestlogreg = grid_search.best_estimator_

Fitting 3 folds for each of 400 candidates, totalling 1200 fits
The accuracy score is 0.8528180354267311 with parameters: {'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 381, 'C': 10}


C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

Accuracy=0.8305085 Precision=0.8541667 Recall=0.9318182 F1=0.8913043


# SVM Classification model with Linear Kernel

In [9]:
svm_lin_model = SVC(kernel="linear", probability=True)
_ = svm_lin_model.fit(X_train, np.ravel(y_train))

In [10]:
# define the performance DataFrame
performance = pd.DataFrame(columns=['model', 'Accuracy', 'Precision', 'Recall', 'F1'])


In [11]:
model_preds = svm_lin_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with linear kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,svm with linear kernel,0.881356,0.877551,0.977273,0.924731


# SVM Classification model with rbf Kernel

In [12]:
svm_rbf_model = SVC(kernel="rbf", C=4, gamma='scale', probability=True)
_ = svm_rbf_model.fit(X_train, np.ravel(y_train))

In [13]:
model_preds = svm_rbf_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with rbf kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,svm with linear kernel,0.881356,0.877551,0.977273,0.924731
0,svm with rbf kernel,0.813559,0.811321,0.977273,0.886598


# SVM Classification model with Polynomial Kernel

In [14]:
svm_poly_model = SVC(kernel="poly", degree=3, coef0=1, C=4, probability=True)
_ = svm_poly_model.fit(X_train, np.ravel(y_train))

In [15]:
model_preds = svm_poly_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

if TP+FP == 0:
    precision = 0
else:
    precision = TP/(TP+FP)

performance = pd.concat([performance, pd.DataFrame({'model':"svm with polynomial kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [precision], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance


,model,Accuracy,Precision,Recall,F1
0,svm with linear kernel,0.881356,0.877551,0.977273,0.924731
0,svm with rbf kernel,0.813559,0.811321,0.977273,0.886598
0,svm with polynomial kernel,0.796610,0.807692,0.954545,0.875000


# SVM Classification model with Randomized Search Kernel


In [ ]:
score_measure = "accuracy"
kfolds = 3
param_grid = {'C': [0.1, 1, 10], 
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['linear','poly','rbf']} 
  
rand_search = RandomizedSearchCV(SVC(), param_grid, refit=True, verbose=3)
  
# fitting the model for randomized search
rand_search.fit(X_train, y_train)
print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_


In [15]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Random search SVM Linear", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,svm with linear kernel,0.881356,0.877551,0.977273,0.924731
0,svm with rbf kernel,0.813559,0.811321,0.977273,0.886598
0,svm with polynomial kernel,0.796610,0.807692,0.954545,0.875000
0,Random search SVM Linear,0.881356,0.877551,0.977273,0.924731


# SVM Classification model with Randomized Search Kernel


In [ ]:
score_measure = "accuracy"
kfolds = 3
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear','poly','rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 75 candidates, totalling 375 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.857 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.963 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.815 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.889 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.704 total time=   0.0s


C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.857 total time= 2.4min


C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 2/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.704 total time= 1.5min


...

C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 2/5] END .......C=1, gamma=1, kernel=linear;, score=0.963 total time=   0.3s
[CV 3/5] END .......C=1, gamma=1, kernel=linear;, score=0.852 total time=   0.0s


C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 4/5] END .......C=1, gamma=1, kernel=linear;, score=0.889 total time=   0.1s
[CV 5/5] END .......C=1, gamma=1, kernel=linear;, score=0.778 total time=   0.0s


C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
c_matrix = confusion_matrix(y_test, grid.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Grid search SVM Linear", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

performance

# Logistic Regression model

In [16]:
log_reg_model = LogisticRegression(penalty='none')
_ = log_reg_model.fit(X_train, np.ravel(y_train))

C:\Users\santo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
df = pd.DataFrame({'model': ['logistic regression', 'SVM', 'decision tree'], 'accuracy': [0.8, 0.75, 0.9]})

# sort the DataFrame by the 'accuracy' column in descending order
df_sorted = df.sort_values(by='accuracy', ascending=False)

# print the sorted DataFrame
print(df_sorted)

                 model  accuracy
2        decision tree      0.90
0  logistic regression      0.80
1                  SVM      0.75


In [27]:
model_preds = log_reg_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"default logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,svm with linear kernel,0.881356,0.877551,0.977273,0.924731
0,svm with rbf kernel,0.813559,0.811321,0.977273,0.886598
0,svm with polynomial kernel,0.796610,0.807692,0.954545,0.875000
0,default logistic,0.830508,0.854167,0.931818,0.891304
0,Logistic Regression Randomised,0.830508,0.854167,0.931818,0.891304
0,Decision tree random search,0.830508,0.904762,0.863636,0.883721
0,Grid search DT,0.847458,0.888889,0.909091,0.898876
0,default logistic,0.830508,0.854167,0.931818,0.891304


# RandomizedSearch with Logistic Regression

In [28]:
score_measure = "accuracy"
LR=LogisticRegression()
kfolds = 5
param_grid = {'C': [0.1, 1, 10,0.001], 
              "solver" : [ 'lbfgs', 'liblinear'],
              "penalty" : ['l1','l2','lasso','elastic']} 
  
grid = RandomizedSearchCV(LR, param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END ..C=1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ..C=1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ..C=1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ..C=1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ..C=1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.857 total time=   0.0s
[CV 2/5] END C=0.1, penalty=l1, solve

C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

In [29]:
model_preds = grid.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression Randomised", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

# Logistic Regression with Grid Search

In [30]:
score_measure = "accuracy"
kfolds = 5
param_grid = {'C': [0.1, 1, 10], 
              'solver' : [ 'lbfgs', 'liblinear'],
              'penalty' : ['l1','l2','lasso','elastic']} 
  
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.857 total time=   0.0s
[CV 2/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.852 total time=   0.0s
[CV 3/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.852 total time=   0.0s
[CV 4/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.852 total time=   0.0s
[CV 5/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.741 total time=   0.0s
[CV 1/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.821 total time=   0.0s
[CV 2/5] END ...C=0.1, penalty=l2, solver=

C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 2/5] END C=10, penalty=l1, solver=liblinear;, score=0.889 total time=   0.0s
[CV 3/5] END C=10, penalty=l1, solver=liblinear;, score=0.852 total time=   0.0s
[CV 4/5] END C=10, penalty=l1, solver=liblinear;, score=0.926 total time=   0.0s
[CV 5/5] END C=10, penalty=l1, solver=liblinear;, score=0.778 total time=   0.0s
[CV 1/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.786 total time=   0.0s
[CV 2/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.889 total time=   0.0s
[CV 3/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.852 total time=   0.0s
[CV 4/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.926 total time=   0.0s
[CV 5/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.778 total time=   0.0s
[CV 1/5] END C=10, penalty=l2, solver=liblinear;, score=0.786 total time=   0.0s
[CV 2/5] END C=10, penalty=l2, solver=liblinear;, score=0.889 total time=   0.0s
[CV 3/5] END C=10, penalty=l2, solver=liblinear;, score=0.852 total time=   0.0s
[CV 4/5] END C=10, penalty=l

C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\santo\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

In [31]:
model_preds = grid.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression Grid", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

# Decision tree model using the randomsearch

In [32]:
score_measure = "accuracy"
kfolds = 3

param_grid = {
    'min_samples_split': np.arange(1,60),  
    'min_samples_leaf': np.arange(1,50),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 200), 
    'max_depth': np.arange(1,50), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=100,
                           scoring=score_measure, verbose=1, n_jobs=-1, return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 3 folds for each of 100 candidates, totalling 300 fits
The best accuracy score is 0.8380032206119162
... with parameters: {'min_samples_split': 51, 'min_samples_leaf': 5, 'min_impurity_decrease': 0.0056, 'max_leaf_nodes': 89, 'max_depth': 38, 'criterion': 'entropy'}


C:\Users\santo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\santo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\santo\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\Users\santo\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 250, in fit
    raise ValueError(
ValueError: min_samples_split must be an integer greater than 1 or a float in (0

In [33]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Decision tree random search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

# Decision tree model using the Gridsearch

In [34]:
score_measure = "accuracy"
kfolds = 3

param_grid = {
    'min_samples_split': np.arange(25,32),  
    'min_samples_leaf': np.arange(3,6),
    'min_impurity_decrease': np.arange(0.0001, 0.0004, 0.0001),
    'max_leaf_nodes': np.arange(194,200), 
    'max_depth': np.arange(15,21), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 3 folds for each of 2268 candidates, totalling 6804 fits
The best accuracy score is 0.8380032206119163
... with parameters: {'criterion': 'entropy', 'max_depth': 15, 'max_leaf_nodes': 194, 'min_impurity_decrease': 0.0002, 'min_samples_leaf': 5, 'min_samples_split': 27}


In [35]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

if TP+FP == 0:
    precision = 0  # or precision = np.nan
else:
    precision = TP/(TP+FP)

performance = pd.concat([performance, pd.DataFrame({'model':"Grid search DT", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [precision], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])


In [40]:
df = pd.DataFrame({'model': ['logistic regression', 'SVM', 'decision tree'], 'accuracy': [0.8, 0.75, 0.9]})

# sort the DataFrame by the 'accuracy' column in descending order
df_sorted = df.sort_values(by='accuracy', ascending=False)

# print the sorted DataFrame
print(df_sorted)

                 model  accuracy
2        decision tree      0.90
0  logistic regression      0.80
1                  SVM      0.75


In [43]:
performance.sort_values(by =['Accuracy'])

,model,Accuracy,Precision,Recall,F1
0,svm with polynomial kernel,0.796610,0.807692,0.954545,0.875000
0,svm with rbf kernel,0.813559,0.811321,0.977273,0.886598
0,default logistic,0.830508,0.854167,0.931818,0.891304
0,Logistic Regression Randomised,0.830508,0.854167,0.931818,0.891304
0,Decision tree random search,0.830508,0.904762,0.863636,0.883721
0,default logistic,0.830508,0.854167,0.931818,0.891304
0,Logistic Regression Randomised,0.830508,0.854167,0.931818,0.891304
0,Logistic Regression Grid,0.830508,0.854167,0.931818,0.891304
0,Decision tree random search,0.830508,0.904762,0.863636,0.883721
0,Grid search DT,0.830508,0.904762,0.863636,0.883721


Upon comparing the performance of various machine learning models on the Parkinson dataset, I found that SVM with a linear kernel achieved the highest accuracy. This result suggests that SVM is a powerful algorithm for accurately classifying Parkinson's disease in patients. Further research may be warranted to investigate the underlying factors contributing to the superior performance of SVM in this particular application.